In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# ! pip install pyyaml h5py

# Estrategia de solucion:
A. Armar un conjunto con todas las peliculas que le puede interesar.
B. Poner en la red y ver que prediccion de rating da
C. Tomar las N que mejor predigan.

### A- El conjunto de peliculas base se toma de:
1. Peliculas mayor rating (almacenado previamente en top_k_movies.csv)
3. Peliculas similares a las que vio y rankeo bien (almacenadas en watched_ranked)
4. Peliculas que vio un usuario similar

### B- Poner en la red y predecir
1. Usar la red hecha en el paso fv_05_Neural_Network.ipyb
2. Poner todas las peliculas en el mismo set: movie_idxs
3. Preparar todos los parametros de entrada a la red
4. Predecir

### C- Tomar las N mejores predicciones
1. Usar cantidad_recomendaciones
2. Ordenar las predicciones por rankin
3. No considerar las que ya vio
4. Almacenar las recomendaciones en processed/recommended.csv

# Parametrizacion
* userId: será el usuario del que querramos recomendarle peliculas
* cantidad_recomendaciones = 10 peliculas se recomiendan
* La fecha de cuando se hace la recomendacion esta basada en la ultima pelicula que se vio de la bd . Se hace la simulacion del dia para que la red pueda inferir diferente dado que como entrada esta el mes. (inputData.timestamp)


In [15]:
# userId = 1: tiene 136 calificaciones, avg: 3.68 - basar en red neuronal para tomar los mejores ratings estimados
# userId = 944: usuario sin calificaciones mostrar top_k_movies.csv
# userId = 310: usuario con menos de 10 calificaciones - usar movie_embedding, mostrar las peliculas similares y completar con top_k_movies.csv
user_id = 310

In [16]:
# Simulo el dia actual para la recomendacion, porque la red podria inferir diferente dado que como entrada esta el mes.
# La fecha esta basada en la ultima pelicula que se vio de la bd
recommendation_timestamp = 893286638

cantidad_recomendaciones = 10

In [17]:
recommendations = {
    'user_id': [user_id],
    'movies': ['']
}

In [18]:
def mostrar_peliculas_delete(peliculas, menos, cantidad=cantidad_recomendaciones):
    for movie in peliculas:
        if (movie not in menos):
            pass
    pass

def mostrar_peliculas_usuario_delete(usuarios, menos, cantidad=cantidad_recomendaciones):
    for user in usuarios:
        for movie in user.movies:
            pass

def mostrar_peliculas_top_delete(menos, cantidad=cantidad_recomendaciones):
        print("Recomendaciones para el usuario:\n")
        n, m = top_movies.shape
        recommended_movies = ""
        for i in range(min(n, cantidad_recomendaciones)):
            row = top_movies.loc[i]
            if (row not in menos):
                movie_show = item[item['movie_id'] == row.movie_id]
                recommended_movies += str(row.movie_id) + "/"
                print(i + 1, movie_show.iloc[0].movie_title)
                if (cantidad <= i):
                    save_recommendations_delete(recommended_movies)

def save_recommendations_delete(recommended_movies):
        recommendations['movies'] += [recommended_movies]
        newRecommendations = pd.DataFrame.from_dict(recommendations)
        newRecommendations.to_csv(PATH_PROCESSED + 'recommended.txt', mode='a', index=False, header=False, sep="|")


# Levanto datos de files

In [19]:
import pandas as pd
import numpy as np
import sys
from datetime import datetime

# datetime object containing current date and time
start = datetime.now()

In [20]:
PATH = 'ml-100k/'
PATH_PROCESSED='processed/'
ratings = pd.read_feather(PATH_PROCESSED + 'joined.fth')

In [21]:
top_movies = pd.read_csv(PATH_PROCESSED +'top_k_movies.csv', sep=",", header=0)

In [22]:
u_unique = ratings.user_id.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = ratings.movie_id.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

In [23]:
ratings.user_id = ratings.user_id.apply(lambda x: user2Idx[x])
ratings.movie_id = ratings.movie_id.apply(lambda x: movie2Idx[x])

In [24]:
PATH_PROCESSED='processed/'
PATH = 'dataset/'

user, occupation = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="|") for fname in ['user', 'occupation']]
info, train = [pd.read_csv(PATH + fname+'.csv', low_memory=False, sep="\t") for fname in ['info', 'train']]
test = pd.read_csv(PATH + 'test.csv', low_memory=False, sep=",")
item = pd.read_csv(PATH + 'item.csv',  low_memory=False, encoding='latin-1', sep="|")

item.rename( columns={'movieId' :'movie_id', 'movieTitle' :'movie_title', 'release date' :'release_date', 'video release date' :'video_release_date', 'IMDb URL' :'imdb_url', 'unknown' :'unknown', 'Action' :'action', 'Adventure' :'adventure', 'Animation' :'animation', 'Children\'s' :'children', 'Comedy' :'comedy', 'Crime' :'crime', 'Documentary' :'documentary', 'Drama' :'drama', 'Fantasy' :'fantasy', 'Film-Noir' :'film-noir', 'Horror' :'horror', 'Musical' :'musical', 'Mystery' :'mystery', 'Romance' :'romance', 'Sci-Fi' :'sci-fi', 'Thriller' :'thriller', 'War' :'war', 'Western' :'western'}, inplace=True)

user.rename( columns={'user id' :'user_id', 'age' :'age', 'gender' :'gender', 'occupation' :'occupation', 'zip code' :'zip_code'}, inplace=True)

train.rename( columns={'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
test.rename( columns={'Id' :'id', 'userId' :'user_id', 'movieId' :'movie_id'}, inplace=True)
#data = train + test
data = train


u_unique = data.user_id.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = data.movie_id.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

# Extraigo datos del usuario seleccionado

In [25]:
inputData = user[user['user_id'] == user_id].iloc[0]

# SOLUCION 1: Caso nuevo usuario

In [26]:
user_ratings = ratings[ratings['user_id'] == user_id]
inputData['rating_mean'] = user_ratings.groupby("user_id")["rating"].transform("mean").iloc[0]
inputData['rating_median'] = user_ratings.groupby("user_id")["rating"].transform("median").iloc[0]
inputData['rating_count'] = user_ratings.groupby("user_id")["rating"].transform("count").iloc[0]
inputData['ageRange'] = user_ratings.ageRange.iloc[0]
inputData['occupation'] = user_ratings.occupation.iloc[0]
inputData['gender'] = user_ratings.gender.iloc[0]

inputData['timestamp'] = 0

inputData['movie_id'] = 0
genre = ['action', 'adventure', 'animation',
         "children", 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
         'film-noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi',
         'thriller', 'war', 'western']
inputData['genre'] = genre
inputData['watched'] = user_ratings.movie_id.to_list()
inputData['watched_ranked'] = user_ratings[user_ratings['rating'] >= inputData['rating_mean']].movie_id.to_list()


In [27]:
if len(ratings[ratings['user_id'] == user_id]) == 0:
    print("Usuario sin votaciones")

    print("Recomendaciones para el usuario:\n")
    n,m = top_movies.shape
    recommended_movies = ""
    for i in range(min(n,cantidad_recomendaciones)):
        row = top_movies.loc[i]
        movie_show = item[item['movie_id'] == row.movie_id]
        recommended_movies += str(row.movie_id) + "/"
        print(i+1,movie_show.iloc[0].movie_title)

    recommendations['movies'] = [recommended_movies]
    newRecommendations = pd.DataFrame.from_dict(recommendations)
    #newRecommendations.to_csv(PATH_PROCESSED + 'recommended.txt', mode='a', index=False, header=False, sep="|")
    #sys.exit(0)

# Preparacion de recomendaciones
Levantar la red

In [28]:
import tensorflow as tf
from keras import Model
import keras.backend as K
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [29]:
max_rating = 5
min_rating = 1
av_rating=3.524453125
n_users = int(ratings.user_id.nunique())
n_movies = int(ratings.movie_id.nunique())

In [30]:
model = tf.keras.models.load_model(PATH_PROCESSED + 'fv_model.h5', custom_objects={"root_mean_squared_error": root_mean_squared_error })
#model.load_weights(PATH_PROCESSED + 'fv_weights.hdf5')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 gender (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 occupation (InputLayer)        [(None, 1)]          0           []                               
                                                                                            

In [31]:
model.compile('adam', loss= 'mean_squared_error', metrics=[root_mean_squared_error])

# Obtengo embeddings

In [32]:
movie_embeddings_layer = model.layers[5]
user_embeddings_layer = model.layers[6]
movie_embeddings_layer.name, user_embeddings_layer.name

('Movie-Embedding', 'User-Embedding')

In [33]:
movie_embeddings_matrix = movie_embeddings_layer.get_weights()[0]
user_embeddings_matrix = user_embeddings_layer.get_weights()[0]
movie_embeddings_matrix.shape, user_embeddings_matrix.shape

((1683, 20), (944, 8))

In [34]:
user_embeddings_matrix

array([[ 2.19576350e-32, -1.62225068e-32, -1.25207722e-32, ...,
        -2.04252708e-32, -6.93971722e-32, -4.33982853e-33],
       [ 2.25388141e-32,  6.86170730e-33,  6.28750255e-34, ...,
         2.11964421e-32, -6.40716597e-32,  1.00525485e-32],
       [-2.14986911e-32,  1.42217419e-32, -1.89437850e-32, ...,
         1.02486886e-32,  1.56902753e-32,  1.00575267e-32],
       ...,
       [ 1.08453613e-02, -8.05182476e-03,  3.20736915e-02, ...,
        -3.10995691e-02, -8.06206279e-03,  2.17886306e-02],
       [ 6.05563037e-02, -4.19338271e-02,  7.03833029e-02, ...,
        -6.24318309e-02, -5.77787384e-02,  6.85254335e-02],
       [ 2.28106715e-02, -9.83318314e-03,  1.00313732e-02, ...,
         2.23245677e-02,  1.23228310e-02, -1.78704169e-02]], dtype=float32)

In [35]:
movie_embeddings_matrix[1:3]

array([[-0.00923054, -0.04325443, -0.00411074,  0.00241773, -0.00786701,
         0.04989532, -0.0167722 , -0.00253602,  0.01196022,  0.06542718,
         0.00255265,  0.01099029, -0.03162089,  0.02188444,  0.03408203,
        -0.0629614 , -0.027903  ,  0.01207565,  0.00351955,  0.02993242],
       [ 0.00346067, -0.0090881 ,  0.00779559,  0.00500195,  0.00567791,
         0.01018028, -0.00224234, -0.00051738,  0.00885171,  0.01310112,
        -0.00797035,  0.00164361, -0.02838109, -0.00228153,  0.00907109,
         0.00523848, -0.00493635, -0.00440257,  0.0044716 ,  0.02011755]],
      dtype=float32)

# Usuarios Similares

Esto sirve si el usuario no tiene peliculas vistas o para completar

In [36]:
from sklearn.neighbors import NearestNeighbors
suggestions_count=30

In [37]:
nbrs_user = NearestNeighbors(n_neighbors=suggestions_count, metric='cosine').fit(user_embeddings_matrix)
neighbors_users = nbrs_user.kneighbors([user_embeddings_matrix[user_id]])
neighbors_users

(array([[0.        , 0.02622306, 0.02708387, 0.0359835 , 0.03948975,
         0.04243946, 0.04597127, 0.04630256, 0.0472126 , 0.05097091,
         0.05169976, 0.05982661, 0.06428862, 0.06639254, 0.06690037,
         0.06780797, 0.06904942, 0.06915396, 0.07092977, 0.07104754,
         0.07105756, 0.0733754 , 0.07404542, 0.08057833, 0.08188099,
         0.08316839, 0.08714199, 0.08715987, 0.08730149, 0.08803898]],
       dtype=float32),
 array([[310, 933, 735, 652, 865, 685, 609, 526, 724, 900, 913, 490, 578,
         833, 435, 869, 852, 635, 797, 773, 380, 509, 430, 405, 538, 937,
         452, 702, 731, 601]]))

In [38]:
# Tomo las peliculas de usuarios similares que tengan mayor ranking que el promedio
recommended_movies_similar_users = []
for usr in neighbors_users[1][0]:
    selected_ratings = ratings[(ratings['user_id'] == usr) & (ratings['rating'] >= inputData['rating_mean'])]
    recommended_movies_similar_users += selected_ratings.movie_id.to_list()
recommended_movies_similar_users

[34,
 973,
 1,
 4,
 6,
 8,
 9,
 270,
 16,
 19,
 328,
 34,
 35,
 330,
 38,
 272,
 332,
 749,
 333,
 46,
 464,
 48,
 49,
 50,
 53,
 339,
 138,
 202,
 340,
 68,
 273,
 71,
 78,
 204,
 83,
 84,
 344,
 538,
 548,
 88,
 90,
 93,
 206,
 346,
 94,
 275,
 96,
 347,
 98,
 207,
 208,
 99,
 551,
 102,
 103,
 507,
 210,
 108,
 192,
 111,
 350,
 351,
 112,
 553,
 213,
 353,
 117,
 120,
 354,
 143,
 149,
 172,
 355,
 358,
 220,
 472,
 233,
 237,
 363,
 240,
 281,
 282,
 482,
 372,
 576,
 291,
 473,
 515,
 579,
 380,
 392,
 404,
 447,
 448,
 492,
 606,
 762,
 611,
 620,
 634,
 809,
 835,
 1070,
 1330,
 1,
 6,
 8,
 10,
 17,
 34,
 57,
 545,
 65,
 66,
 67,
 68,
 97,
 118,
 139,
 123,
 141,
 133,
 145,
 146,
 147,
 152,
 153,
 155,
 158,
 279,
 162,
 193,
 164,
 173,
 175,
 195,
 984,
 509,
 180,
 577,
 579,
 435,
 721,
 474,
 957,
 535,
 890,
 651,
 337,
 123,
 145,
 149,
 153,
 162,
 562,
 565,
 196,
 180,
 234,
 323,
 498,
 535,
 685,
 6,
 504,
 336,
 138,
 64,
 781,
 91,
 209,
 123,
 141,
 132,
 159,


# Peliculas Similares

Puede servir si el usuario tiene pocas peliculas vistas. Basadas en las que vio

### Nearest Neighbors

In [39]:
nbrs_movies = NearestNeighbors(n_neighbors=suggestions_count, metric='cosine').fit(movie_embeddings_matrix)

In [40]:
neighbors_movies = nbrs_movies.kneighbors([movie_embeddings_matrix[2]]) # movie_id?

In [41]:
# Tomo las peliculas similares a las que vio con buen promedio
recommended_movies_similar_watched = []

for watched_movie_id in inputData.watched_ranked:
    for mov in nbrs_movies.kneighbors([movie_embeddings_matrix[watched_movie_id]])[1][0]:
        recommended_movies_similar_watched += [mov]
recommended_movies_similar_watched

[34,
 503,
 62,
 355,
 766,
 306,
 741,
 920,
 358,
 981,
 90,
 485,
 105,
 775,
 579,
 894,
 473,
 207,
 869,
 31,
 270,
 115,
 129,
 765,
 427,
 95,
 272,
 711,
 211,
 483,
 973,
 88,
 421,
 1331,
 273,
 251,
 21,
 115,
 1236,
 91,
 741,
 981,
 485,
 501,
 31,
 716,
 355,
 1358,
 306,
 920,
 354,
 242,
 378,
 66,
 563,
 889,
 207,
 758,
 7,
 90]

# Preparar input de la red con peliculas a sugerir

In [42]:
# No se usaran todas las peliculas para recomendar, asi acortamos el unierso
#all_movie_idxs = np.linspace(1, ratings.movie_id.max(), n_movies, dtype=int)

# Unimos las distintas peliculas a recomendar
all_movie_idxs = recommended_movies_similar_watched + recommended_movies_similar_users + top_movies.movie_id.to_list()
all_movie_idxs = set(all_movie_idxs)
n_movies = len(all_movie_idxs)
all_movie_idxs

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 25,
 26,
 27,
 29,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 62,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 75,
 76,
 78,
 80,
 82,
 83,
 84,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 124,
 126,
 127,
 129,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 177,
 178,
 179,
 180,
 182,
 184,
 185,
 186,
 187,
 188,
 192,
 193,
 194,
 195,
 196,
 197,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 

In [43]:
# Preparar la entrada a la red: Datos de pelicula
releaseYears = []
watchedReleaseDiffs = []
genres = ratings[ratings['movie_id'] == -1].head(1)[genre]
movie_idxs = []
for movie in all_movie_idxs:
    result = ratings[ratings['movie_id'] == movie]
    if (len(result) > 0):
        releaseYears += [result.ReleaseYear.values[0]]
        watchedReleaseDiffs += [result.watchReleaseDiff.values[0]]
        genres = genres.append(result.head(1)[genre])
        movie_idxs += [result.movie_id.values[0]]

In [44]:
# computes the max timestamp to be considered as today
inputData.timestamp = ratings.timestamp.max()

In [45]:
# Preparar la entrada a la red: Datos de usuario
n_movies = n_movies - 1
user_idxs = np.zeros(n_movies, dtype=int) + user_id
user_genders = np.zeros(n_movies, dtype=int) + inputData.gender
user_occupations = np.zeros(n_movies, dtype=int) + inputData.occupation
timestamps = np.zeros(n_movies, dtype=int) + inputData.timestamp
ageRanges = np.zeros(n_movies, dtype=int) + inputData.ageRange

# Verify

In [46]:
inputs_val = [user_idxs, movie_idxs, timestamps, genres,
              user_genders, user_occupations, watchedReleaseDiffs, ageRanges, releaseYears]

In [47]:
# Convert input data into series
for i in range(len(inputs_val)):
    if (i != 3):
        inputs_val[i] = pd.Series(inputs_val[i])

In [48]:
# Verify input values
for i in range(len(inputs_val)):
    if (i == 3):
        print(i, type(inputs_val[i]))
    else:
        print(i, type(inputs_val[i]), len(inputs_val[i]), type(inputs_val[i][0]), inputs_val[i][0])

0 <class 'pandas.core.series.Series'> 704 <class 'numpy.int64'> 310
1 <class 'pandas.core.series.Series'> 704 <class 'numpy.int64'> 1
2 <class 'pandas.core.series.Series'> 704 <class 'numpy.float64'> 1.82796423103278
3 <class 'pandas.core.frame.DataFrame'>
4 <class 'pandas.core.series.Series'> 704 <class 'numpy.int64'> 1
5 <class 'pandas.core.series.Series'> 704 <class 'numpy.int64'> 3
6 <class 'pandas.core.series.Series'> 704 <class 'numpy.float64'> -0.5019160586050823
7 <class 'pandas.core.series.Series'> 704 <class 'numpy.int64'> 2
8 <class 'pandas.core.series.Series'> 704 <class 'numpy.float64'> -135.55962914359867


In [49]:
predictions = model.predict(inputs_val)

22/22 [==============================] - 0s 1ms/step


In [50]:
print(predictions.max(), predictions.min())
predictions

3.5273888 3.5273888


array([[3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],
       [3.5273888],


In [51]:
sortedMovies = np.argsort(predictions[:,0])[::-1]
len(sortedMovies)

704

In [52]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

In [53]:
# Finalmente obtener los nombres de las peliculas a recomendar y almacenarlo
import csv

recommended_movies = []
i = 0
for movie in sortedMovies:
    movie_show = item[item['movie_id'] == movie]
    if (len(movie_show) > 0):
        if (movie not in inputData['watched']):
            recommended_movies += [str(movie_show.iloc[0].movie_title)]
            # print(i+1,movie_show.iloc[0].movie_title)
            i += 1
    if i >= cantidad_recomendaciones:
        break

recommendations['movies'] = recommended_movies

header = str(len(recommended_movies)) + " Peliculas Recomendadas para el Usuario: " + str(user_id)
print(dt_string, "-", header)
with open(PATH_PROCESSED + 'recommended.txt', 'a') as recfile:
    recfile.writelines("\n")
    recfile.writelines(dt_string)
    recfile.writelines("\n")
    recfile.writelines(header)
    recfile.writelines("\n")
    idx = 1
    for title in recommended_movies:
        recfile.writelines(str(idx) + ". " + title)
        recfile.writelines("\n")
        idx += 1

print(recommended_movies)

31/12/2022 10:02:17 - 10 Peliculas Recomendadas para el Usuario: 310
["Widows' Peak (1994)", 'Barcelona (1994)', 'Sneakers (1992)', 'Raising Arizona (1987)', 'Jerry Maguire (1996)', 'Citizen Ruth (1996)', 'Mars Attacks! (1996)', 'Jaws (1975)', 'Under Siege (1992)', 'Young Guns (1988)']


In [54]:
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
print("Elapsed", now - start)

date and time = 31/12/2022 10:02:17
Elapsed 0:00:12.179888
